In [59]:
import pandas as pd
import joblib




In [60]:
# 1️⃣ Загружаем тестовые данные
test_df = pd.read_excel("data/test.xlsx")

# 2️⃣ Загружаем обученную модель
model = joblib.load("RandomForestRegressor.pkl")  # Или другой файл модели

# 3️⃣ Определяем список признаков, которые использовались при обучении
feature_columns = model.feature_names_in_  # Получаем признаки, которые ожидает модель

In [61]:
print("Ожидаемые признаки:", feature_columns)
print("Колонки test_df:", test_df.columns)


Ожидаемые признаки: ['year' 'month' 'day' 'week' 'Price_source' 'Price_Diff']
Колонки test_df: Index(['dt', 'Цена на арматуру'], dtype='object')


In [62]:
if 'dt' in test_df.columns:
    test_df["dt"] = pd.to_datetime(test_df["dt"])
    test_df["year"] = test_df["dt"].dt.year
    test_df["month"] = test_df["dt"].dt.month
    test_df["day"] = test_df["dt"].dt.day
    test_df["week"] = test_df["dt"].dt.isocalendar().week

if 'Цена на арматуру' in test_df.columns:
    test_df["Price_source"] = test_df["Цена на арматуру"].shift(1)
    test_df["Price_Diff"] = test_df["Цена на арматуру"].diff()

    # Заполняем NaN средними значениями соседних строк
    test_df["Price_source"] = test_df["Price_source"].interpolate(method='linear')
    test_df["Price_Diff"] = test_df["Price_Diff"].interpolate(method='linear')

    # Если остались NaN (в начале/конце), заменяем их средним значением
    test_df["Price_source"] = test_df["Price_source"].fillna(test_df["Price_source"].mean())
    test_df["Price_Diff"] = test_df["Price_Diff"].fillna(test_df["Price_Diff"].mean())

In [69]:
feature_columns = model.feature_names_in_
print("Ожидаемые признаки:", feature_columns)
print("Колонки test_df:", test_df.columns)

Ожидаемые признаки: ['year' 'month' 'day' 'week' 'Price_source' 'Price_Diff']
Колонки test_df: Index(['dt', 'Цена на арматуру', 'year', 'month', 'day', 'week',
       'Price_source', 'Price_Diff', 'Прогноз_цены', 'Закупка_недель'],
      dtype='object')


In [64]:
X_test = test_df[feature_columns]
test_df["Прогноз_цены"] = model.predict(X_test)

# 5️⃣ Определяем N (количество недель закупки)
def calculate_weeks(row):
    if row["Прогноз_цены"] > row["Цена на арматуру"] * 1.02:  # Цена растёт (+2%)
        return 6
    elif row["Прогноз_цены"] > row["Цена на арматуру"]:  # Незначительный рост
        return 4
    elif row["Прогноз_цены"] < row["Цена на арматуру"] * 0.98:  # Цена падает (-2%)
        return 1
    else:
        return 3  # Стабильная цена → средний объём закупки

test_df["Закупка_недель"] = test_df.apply(calculate_weeks, axis=1)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.0s


In [65]:
X_test = test_df[feature_columns]

In [66]:
test_df["Прогноз_цены"] = model.predict(X_test).round().astype(int)
test_df = test_df.drop(columns=["Прогноз_недель"], errors="ignore")

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.0s


In [68]:
test_df.head()

,dt,Цена на арматуру,year,month,day,week,Price_source,Price_Diff,Прогноз_цены,Закупка_недель
0,2022-09-05,46000,2022,9,5,36,43833.333333,351.851852,43938,1
1,2022-09-12,46000,2022,9,12,37,46000.000000,0.000000,45981,3
2,2022-09-19,46000,2022,9,19,38,46000.000000,0.000000,45748,3
3,2022-09-26,45500,2022,9,26,39,46000.000000,-500.000000,46355,4
4,2022-10-03,45500,2022,10,3,40,45500.000000,0.000000,46010,4
